In [3]:
### Import the necessary libraries ###
import os
import cv2
import pickle
import numpy as np
import face_recognition as fr

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import backend as K

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
### Load in the dataset that we want the test both models on ###
import os
import cv2
import numpy as np
import face_recognition

DATA_DIR = './data'
WIDTH, HEIGHT, CHANNELS = 170, 170, 3

faces, labels = [], []

# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

''' ==================================================================================================== '''

def detect_face(img):
    ### take an image and return face location(s) ###
    location = face_recognition.face_locations(img)
    
    if(len(location) == 0):
        return None
    
    top, right, bottom, left = location[0]
    return (left, top, right, bottom)

def preprocessing(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (170, 170))

    ### Standardize the image ###
    mean = np.mean(image)
    std = np.std(image)
    image = (image - mean)/std

    return image

count = 0
source_id = 0
record_file = 'images.tfrecord'
width, height, channels = 128,128,3
with tf.io.TFRecordWriter(record_file) as writer:
    for (dir_, dirs, files) in os.walk(DATA_DIR):
        file_count = 0
        for file_ in files:
            file_count += 1
            count += 1

            if(file_count >= 30):
                continue

            abs_path = os.path.join(dir_, file_)

            # print('File %s processed ' % abs_path)
            feature = {
                'image/filename' : _bytes_feature(abs_path.encode()),
                'image/encoded' : _bytes_feature(open(abs_path, 'rb').read()),
                'image/source_id' : _int64_feature(source_id)
            }

            tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(tf_example.SerializeToString())
        
    source_id += 1

print(count)
faces = np.array(faces)
labels = np.array(labels)

572


In [7]:
# Create a dictionary describing the features.
image_feature_description = {
    'image/filename': tf.io.FixedLenFeature([], tf.string),
    'image/encoded': tf.io.FixedLenFeature([], tf.string),
    'image/source_id': tf.io.FixedLenFeature([], tf.int64)
}

def _parse_image_function(example_proto):
    # Parse the input tf.train.Example proto using the dictionary above.
    x = tf.io.parse_single_example(example_proto, image_feature_description)
    x_train = tf.image.decode_jpeg(x['image/encoded'], channels=3)
    x_train = tf.image.resize(x_train, (width, height))
    y_train = tf.one_hot(x['image/source_id'], 5000)
        
    return (x_train, y_train), y_train

### Parse the tfrecord to a training dataset ###
def load_tfrecord_dataset(tfrecord_name, batch_size, shuffle=True, buffer_size=400000):
    """load dataset from tfrecord"""
    print('Getting dataset length')
    dataset_len = 572 #int(sum(1 for _ in tf.data.TFRecordDataset(tfrecord_name)))
    print(dataset_len / batch_size)
    train_size = int(0.85 * (dataset_len / batch_size))
    print(train_size)
    val_size = int(0.15 * (dataset_len / batch_size))
    print(val_size)

    raw_dataset = tf.data.TFRecordDataset(tfrecord_name)
    raw_dataset = raw_dataset.repeat()

    if shuffle:
        raw_dataset = raw_dataset.shuffle(buffer_size=buffer_size)

    dataset = raw_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    print('Splitting train - val')
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)

    train_dataset = train_dataset.repeat()
    val_dataset = val_dataset.repeat()

    return train_dataset, val_dataset

print('Getting train and test datasets .. ')
train_dataset, test_dataset = load_tfrecord_dataset('images.tfrecord', 32)

Getting train and test datasets .. 
Getting dataset length
17.875
15
2
Splitting train - val


In [18]:
iterator = train_dataset.make_initializable_iterator()
batch = iterator.get_next()

(trainX, trainY), trainY = batch
trainX.numpy()

AttributeError: 'Tensor' object has no attribute 'numpy'